## Generates a TV Listing table for a single day

In [ ]:
# ============================ SINGLE-DAY SPORTS GRID ============================
# Creates a one-day, merged-cells TV listings table with per-sport fill + text styles
# Stacks overlapping events into overflow rows per channel.
# ===============================================================================

import os
from datetime import date, datetime, timedelta, time as dtime
from zoneinfo import ZoneInfo
import math
import re
import pandas as pd
import requests
import xlsxwriter

# ----------------------------- CONFIG ------------------------------------------
OUTPUT_DIR   = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
LOCAL_TZ     = ZoneInfo("America/Detroit")

# Choose the day and time window you want to display
DAY_DATE     = date(2025, 11, 1)            # the day to render
SLOT_MIN     = 30                            # minutes per column (30 is common)
WINDOW_START = dtime(12, 0)                  # 12:00 local (Noon)
WINDOW_END   = dtime(0, 30)                  # end boundary next day (e.g., 12:30 AM)
# NOTE: WINDOW_END is treated as possibly on the *next* day if earlier than start.


### OUPUT FILE NAME
out_xlsx = os.path.join(OUTPUT_DIR, f"SingleDay_{DAY_DATE.isoformat()}.xlsx")

# Channel-number profile (sports-relevant)
CHANNEL_MAP = {
    # Locals
    "CBS": 3, "NBC": 4, "FOX": 6, "ABC": 7,
    # Sports tier
    "ESPN": 11, "ESPNews": 12, "ESPNU": 13, "ESPN2": 14,
    "FS1": 15, "Big Ten": 47, 
    ## Getting rid of Gold Channel for now because can't grab listings from ESPN API
    # "Golf": 52,
    # Entertainment with sports
    "USA": 18, "TNT": 19, "truTV": 20, "TBS": 21,
}

# ESPN broadcast alias normalization (add as needed)
CHANNEL_ALIASES = {
    "abc": "ABC", "abc network": "ABC",
    "fox": "FOX", "fox network": "FOX",
    "cbs": "CBS", "cbs network": "CBS",
    "nbc": "NBC", "nbc network": "NBC", "nbc sports": "NBC", "nbcsn": "NBC",

    "espn": "ESPN", "espn2": "ESPN2", "espnu": "ESPNU",
    "espn news": "ESPNews", "espnnews": "ESPNews",

    "fs1": "FS1", "fox sports 1": "FS1", "fox sports1": "FS1",

    "btn": "Big Ten", "big ten network": "Big Ten",
    "golf channel": "Golf",

    "usa": "USA", "usa network": "USA",
    "tnt": "TNT", "tnt hd": "TNT",
    "tbs": "TBS",
    "trutv": "truTV",
}
EXCLUDE_STREAMING_KEYS = ("espn+", "espn plus", "espn app", "peacock+", "peacock premium")

# Select sports to include in the listing - Comment / uncomment as desired
SPORTS = [
    ("NFL", ["football/nfl"]),
    ("NCAA Football", ["football/college-football"]),
    ("NBA", ["basketball/nba"]),
    ("NHL", ["hockey/nhl"]),
    ("NCAA Hockey", ["hockey/mens-college-hockey"]),
    ("MLB", ["baseball/mlb"]),
    
    ("MBB", ["basketball/mens-college-basketball"]),
    ("WBB", ["basketball/womens-college-basketball"]),

    ### All Known Available Leagues (uncomment to enable more)
    # ("NCAA Baseball", ["baseball/college-baseball"]),
    # ("NCAA Lacrosse(M)", ["lacrosse/mens-college-lacrosse"]),
    ("NCAA Soccer(M)", ["soccer/usa.ncaa.m.1"]),
    # ("NCAA Volleyball(M)", ["volleyball/mens-college-volleyball"]),
    # ("NCAA Water Polo(M)", ["waterpolo/mens-college-water-polo"]),
    ("NCAA Volleyball(W)", ["volleyball/womens-college-volleyball"]),
    # ("NCAA Softball", ["softball/college-softball"]),
    # # Field Hockey
    # ("NCAA Field Hockey", ["fieldhockey/womens-college-field-hockey"]),
    # # Womens Ice Hockey
    # ("NCAA Ice Hockey(W)", ["hockey/womens-college-hockey"]),
    # # Lacrosse (W)
    # ("NCAA Lacrosse(W)", ["lacrosse/womens-college-lacrosse"]),
    # # Soccer Women
    # ("NCAA Soccer(W)", ["soccer/usa.ncaa.w.1"]),
    # # Volleyball (W)
    # ("NCAA Volleyball(W)", ["volleyball/womens-college-volleyball"]),
    # # Water Polo (W)
    # ("NCAA Water Polo(W)", ["waterpolo/womens-college-water-polo

    # # Soccer
    ("MLS",   ["soccer/usa.1"]),
    ("EPL",   ["soccer/eng.1"]),
    # ("UCL",   ["soccer/uefa.champions"]),
    # # La Liga
    # ("La Liga", ["soccer/esp.1"]),
    # # Bundesliga
    # ("Bundesliga", ["soccer/ger.1"]),
    # # Nation Womens NWSL
    # ("NWSL",  ["soccer/usa.nwsl"]),
    # # Liga MX
    # ("Liga MX", ["soccer/mex.1"]),
]

# Human-friendly names for the keys you put in STYLE
DISPLAY_NAME = {
    "football/nfl": "NFL",
    "football/college-football": "NCAA Football",
    "basketball/nba": "NBA",
    "basketball/mens-college-basketball": "Men’s College Basketball",
    "basketball/womens-college-basketball": "Women’s College Basketball",
    "hockey/nhl": "NHL",
    "hockey/college-hockey": "NCAA Hockey",
    "baseball/mlb": "MLB",
    # add more if you add to STYLE
}


# ---------------------------- STYLE GUIDE --------------------------------------
STYLE = {
    # ---------------- PRO ----------------
    "basketball/nba": {
        "bg": "#FF7A00", "font_color": "#FFFFFF", "bold": False,  "prefix": "NBA",
        "est_minutes": 150,
    },
    "hockey/nhl": {
        "bg": "#3366ff", "font_color": "#FFFFFF", "bold": False,  "prefix": "NHL",
        "est_minutes": 150,
    },
    "football/nfl": {
        "bg": "#800000", "font_color": "#FFFFFF", "bold": False,  "prefix": "NFL",
        "est_minutes": 195,
    },
    "baseball/mlb": {
        "bg": "#000080", "font_color": "#FFFFFF", "bold": True,  "prefix": "MLB",
        "est_minutes": 180,
    },

    # ---------------- COLLEGE ----------------
    "hockey/college-hockey": {
        "bg": "#A9CCE3", "font_color": "#1B1B1B", "bold": False,  "prefix": "NCAA Hockey",
        "est_minutes": 150,
    },
    # muted gold/yellow
    "football/college-football": {
        "bg": "#F4E38A", "font_color": "#1B1B1B", "bold": False,  "prefix": "CFB",
        "est_minutes": 210,
    },
    "basketball/mens-college-basketball": {
        "bg": "#ffa34d", "font_color": "#1B1B1B", "bold": False,  "prefix": "NCAA (M)",
        "est_minutes": 135,
    },
    "basketball/womens-college-basketball": {
        "bg": "#ffca99", "font_color": "#1B1B1B", "bold": False,  "prefix": "NCAA (W)",
        "est_minutes": 120,
    },
    # Men's college hockey
    "hockey/mens-college-hockey": {
        "bg": "#ccd9ff", "font_color": "#1B1B1B", "bold": True,  "prefix": "NCAA Hockey",
        "est_minutes": 150,
    },
}

# ---- Special styling rules (colors/fonts you can tweak) ----------------------
MSU_STYLE = {"bg": "#0B3D2E", "font_color": "#FFFFFF", "bold": True, "font_size": 16, "thick_border": True}
B1G_STYLE = {"bg": "#A6FFFF", "font_color": "#1B1B1B", "bold": True}     # muted red/orange family
RANKED_STYLE = {"bg": "#ff0000", "font_color": "#FFFFFF", "bold": True}  # Bright Red for Top-25



# Big Ten schools by school/location name as ESPN usually reports them.
B1G_SCHOOLS = {
    # Originals + expansions (locations, not nicknames)
    "illinois", "indiana", "iowa", "maryland", "michigan", "michigan state",
    "minnesota", "nebraska", "northwestern", "ohio state", "penn state",
    "purdue", "rutgers", "wisconsin",
    # 2024 additions
    "ucla", "usc", "oregon", "washington",
}

DEFAULT_STYLE = {"bg": "#DDDDDD", "font_color": "#000000", "bold": False, "prefix": "", "est_minutes": 120}

# -------------------------- FETCH & NORMALIZE ----------------------------------
def normalize_key(s: str) -> str:
    return re.sub(r"[^a-z0-9+ ]", "", s.lower()).strip()

def normalize_channel_name(name: str) -> str | None:
    if not name:
        return None
    lk = name.lower()
    if any(x in lk for x in EXCLUDE_STREAMING_KEYS):
        return None
    return CHANNEL_ALIASES.get(normalize_key(name))

def team_label(c: dict) -> str:
    t = (c or {}).get("team", {}) or {}
    rank = (c or {}).get("curatedRank", {}).get("current")
    nm = t.get("displayName") or t.get("shortDisplayName") or t.get("name") or ""
    return (f"#{rank} " if rank and rank != 99 else "") + nm

##############
def _names_for_competitors(comp: dict) -> list[dict]:
    """Return the raw competitor dicts and a lower-cased set of name tokens for matching."""
    comps = comp.get("competitors", []) or []
    tokens = set()
    for c in comps:
        t = (c or {}).get("team", {}) or {}
        for k in ("location", "displayName", "shortDisplayName", "name", "abbreviation"):
            v = (t.get(k) or "").lower()
            if v:
                tokens.add(v)
    return comps, tokens

def comp_has_msu(comp: dict) -> bool:
    _, toks = _names_for_competitors(comp)
    return any(x in toks for x in {"michigan state", "michigan st", "msu"})

def comp_has_big_ten(comp: dict) -> bool:
    comps, _ = _names_for_competitors(comp)
    for c in comps:
        loc = ((c.get("team") or {}).get("location") or "").lower()
        if loc in B1G_SCHOOLS:
            return True
    return False

def comp_has_top25(comp: dict) -> bool:
    comps, _ = _names_for_competitors(comp)
    for c in comps:
        r = (c.get("curatedRank") or {}).get("current")
        if isinstance(r, int) and 1 <= r <= 25:
            return True
    return False

def style_for_comp(base_style: dict, comp: dict, league_key: str) -> tuple[dict, str]:
    """
    Apply priority overrides: MSU > Big Ten > Top-25 > base.
    Returns (style_dict, legend_label).
    """
    if comp_has_msu(comp):
        return ({**base_style, **MSU_STYLE}, "Michigan State")
    if comp_has_big_ten(comp):
        return ({**base_style, **B1G_STYLE}, "Big Ten")
    if comp_has_top25(comp):
        return ({**base_style, **RANKED_STYLE}, "Top-25")

    # fall back to the league’s display name for base styles
    label = DISPLAY_NAME.get(league_key, league_key)
    return (base_style, label)

# def style_for_comp(base_style: dict, comp: dict) -> dict:
#     """Apply priority overrides: MSU > Big Ten > Top 25 > base."""
#     if comp_has_msu(comp):
#         return {**base_style, **MSU_STYLE}
#     if comp_has_big_ten(comp):
#         return {**base_style, **B1G_STYLE}
#     if comp_has_top25(comp):
#         return {**base_style, **RANKED_STYLE}
#     return base_style


def build_title(comp: dict, prefix_womens=False) -> str:
    comps = comp.get("competitors", []) or []
    by_side = {c.get("homeAway"): c for c in comps}
    away = team_label(by_side.get("away", {}))
    home = team_label(by_side.get("home", {}))
    title = f"{away} at {home}".strip()
    if prefix_womens:
        title = f"(W) {title}"
    return title

def to_local(iso_str: str):
    if not iso_str:
        return None
    ts_utc = pd.to_datetime(iso_str, errors="coerce", utc=True)
    if pd.isna(ts_utc):
        return None
    return ts_utc.tz_convert(LOCAL_TZ).tz_localize(None)

def _hm(ts):
    # cross-platform "h:mma/p" without leading zero
    try:
        return ts.strftime("%-I:%M%p").lower()
    except ValueError:
        return ts.strftime("%#I:%M%p").lower()


def fetch_day_sport_multi(day: date, league_keys: list[str]) -> list[dict]:
    ymd = day.strftime("%Y%m%d")
    last_err = None
    for k in league_keys:
        url = f"https://site.api.espn.com/apis/site/v2/sports/{k}/scoreboard?dates={ymd}"
        try:
            r = requests.get(url, timeout=20)
            r.raise_for_status()
            return r.json().get("events", []) or []
        except Exception as e:
            last_err = e
            continue
    if last_err:
        print(f"[WARN] {day} {'/'.join(league_keys)} fetch failed: {last_err}")
    return []

def extract_broadcast_tokens(ev: dict, comp: dict) -> list[str]:
    tokens, seen = [], set()
    cand = []
    for b in (comp.get("broadcasts") or []):
        media = b.get("media") or {}
        for k in ("shortName","name"):
            if media.get(k): cand.append(str(media[k]))
        for k in ("shortName","name"):
            if b.get(k): cand.append(str(b[k]))
        for n in (b.get("names") or []):
            cand.append(str(n))
    for gb in (ev.get("geoBroadcasts") or []):
        chan = gb.get("media", {}).get("shortName") or gb.get("media", {}).get("name")
        if chan: cand.append(str(chan))
    b = comp.get("broadcast") or {}
    if isinstance(b, dict):
        for k in ("shortName","name"):
            if b.get(k): cand.append(str(b[k]))
    for raw in cand:
        s = re.sub(r"[\/&]| and ", ",", raw, flags=re.IGNORECASE)
        for tok in [p.strip() for p in s.split(",") if p.strip()]:
            nk = normalize_key(tok)
            if nk and nk not in seen:
                seen.add(nk)
                tokens.append(tok)
    return tokens

def style_for_league(league_key: str) -> dict:
    return STYLE.get(league_key, DEFAULT_STYLE)

# --------------------- BUILD TIME GRID STRUCTURE -------------------------------
def window_bounds(day: date):
    start_dt = datetime.combine(day, WINDOW_START, tzinfo=LOCAL_TZ).replace(tzinfo=None)
    end_day = day if WINDOW_END >= WINDOW_START else day + timedelta(days=1)
    end_dt = datetime.combine(end_day, WINDOW_END, tzinfo=LOCAL_TZ).replace(tzinfo=None)
    return start_dt, end_dt

def build_time_columns(start_dt: datetime, end_dt: datetime, slot_minutes: int):
    labels = []
    stamps = []
    cur = start_dt
    while cur < end_dt:
        stamps.append(cur)
        try:
            lab = cur.strftime("%-I:%M").lstrip("0")
        except ValueError:
            lab = cur.strftime("%#I:%M").lstrip("0")
        if lab in ("12:00", "0:00"):
            lab = "Noon" if cur.hour == 12 else lab
        labels.append(lab)
        cur += timedelta(minutes=slot_minutes)
    return stamps, labels

# --------------------- GATHER EVENTS (single day) ------------------------------
START_DT, END_DT = window_bounds(DAY_DATE)
time_stamps, COL_LABELS = build_time_columns(START_DT, END_DT, SLOT_MIN)

rows = []
for sport_label, league_keys in SPORTS:
    events = fetch_day_sport_multi(DAY_DATE, league_keys)
    if not events:
        continue
    for ev in events:
        for comp in (ev.get("competitions") or []):
            start_local = to_local(comp.get("date") or ev.get("date"))
            if start_local is None:
                continue
            if start_local < START_DT or start_local >= END_DT:
                continue

            # channels (filter to our lineup)
            chans = set()
            for tok in extract_broadcast_tokens(ev, comp):
                canon = normalize_channel_name(tok)
                if canon and canon in CHANNEL_MAP:
                    chans.add(canon)
            if not chans:
                continue  # for single-day TV table, ignore streaming-only here

            # display text with ranks
            is_womens = "womens" in (league_keys[0] or "")
            title = build_title(comp, prefix_womens=is_womens)

            league_key_for_style = next((k for k in league_keys if k in STYLE), league_keys[0])
            base_st = style_for_league(league_key_for_style)
            # Apply overrides AND get legend label for this event
            st, legend_label = style_for_comp(base_st, comp, league_key_for_style)
            minutes = st.get("est_minutes", DEFAULT_STYLE["est_minutes"])



            # slot math
            start_idx = max(0, math.floor((start_local - START_DT).total_seconds() / 60 / SLOT_MIN))
            span = max(1, math.ceil(minutes / SLOT_MIN))
            end_idx = min(len(time_stamps), start_idx + span)

            # build label
            prefix = st.get("prefix", "")
            # --- tag text for the first line ---
            # Prefer the style prefix (e.g., "MLB", "NFL"); fall back to league display name
            tag_text = (st.get("prefix") or DISPLAY_NAME.get(league_key_for_style, league_key_for_style))

            # --- time string from local start time ---
            hm = _hm(start_local)  # "8pm" or "8:30pm"

            # --- two-line cell text ---
            # Line 1: "8pm (MLB)"
            # Line 2: "Team A at Team B"
            cell_text = f"{hm} ({tag_text})\n{title}"


            for ch in chans:
                rows.append({
                    "channel": ch,
                    "channel_num": CHANNEL_MAP[ch],
                    "start_idx": start_idx,
                    "end_idx": end_idx,
                    "text": cell_text,
                    "style": st,
                    "legend_label": legend_label,   # <— NEW
                })


df = pd.DataFrame(rows)
df = df.sort_values(["channel_num", "start_idx"]).reset_index(drop=True)

# ------------------ NEW: PACK OVERLAPS INTO OVERFLOW ROWS ----------------------
# For each channel, greedily place events into non-overlapping "tracks".
# Track 0 = main row, track 1.. = overflow rows for that channel.

from collections import defaultdict

events_by_channel = defaultdict(list)
for r in df.itertuples(index=False):
    events_by_channel[(r.channel, r.channel_num)].append(
        dict(start_idx=int(r.start_idx), end_idx=int(r.end_idx), text=r.text, style=r.style)
    )

def pack_tracks(evts):
    """Return list of tracks, each a list of event dicts (no overlaps within a track)."""
    evts = sorted(evts, key=lambda e: (e["start_idx"], e["end_idx"]))
    tracks = []         # list[list[event]]
    last_ends = []      # track -> last end_idx
    for e in evts:
        placed = False
        for t_idx, last_end in enumerate(last_ends):
            if e["start_idx"] >= last_end:      # ok to place (end is exclusive)
                tracks[t_idx].append(e)
                last_ends[t_idx] = e["end_idx"]
                placed = True
                break
        if not placed:
            tracks.append([e])
            last_ends.append(e["end_idx"])
    return tracks

# Build channel order and their tracks
ordered_channels = sorted(CHANNEL_MAP.items(), key=lambda x: x[1])  # [(name,num), ...]
channel_tracks = []  # [(channel_name, channel_num, [track0, track1, ...])]
for ch_name, ch_num in ordered_channels:
    tracks = pack_tracks(events_by_channel.get((ch_name, ch_num), []))
    channel_tracks.append((ch_name, ch_num, tracks))

# --------------------------- WRITE EXCEL (merged cells) ------------------------
# import xlsxwriter

# out_xlsx = os.path.join(OUTPUT_DIR, f"SingleDay_{DAY_DATE.isoformat()}.xlsx")
wb = xlsxwriter.Workbook(out_xlsx)
ws = wb.add_worksheet("Day")

# Layout columns:
num_col, chan_col, first_time_col = 0, 1, 2
last_time_col = first_time_col + len(COL_LABELS) - 1

# Formats
title_fmt = wb.add_format({"bold": True, "font_size": 48, "align": "center", "valign": "vcenter"})
header_fmt = wb.add_format({"bold": True, "font_size": 12, "align": "center", "valign": "vcenter", "border": 1, "bg_color": "#FFF4CC"})
num_hdr = wb.add_format({"bold": True, "font_size": 24, "align": "center", "valign": "vcenter"})
chan_hdr = wb.add_format({"bold": True, "font_size": 20, "align": "center", "valign": "vcenter"})
# Subtle border for *empty* cells
slot_border = wb.add_format({
    "border": 1,                  # thin
    "border_color": "#DDDDDD",    # light gray
    "align": "center",
    "valign": "vcenter",
})

# Channel label formats (alternating row shading)
row_even = wb.add_format({
    "align": "center",
    "valign": "vcenter",
    "border": 1,
    "font_size": 24,       # <--- bump text size here
    "bold": True,
})
row_odd  = wb.add_format({
    "align": "center",
    "valign": "vcenter",
    "border": 1,
    "bg_color": "#F5F5F5",
    "font_size": 24,
    "bold": True,
})

num_even = wb.add_format({
    "align": "center",     # <--- was right-align; now centered
    "valign": "vcenter",
    "border": 1,
    "font_size": 24,
    "bold": True,
})
num_odd  = wb.add_format({
    "align": "center",
    "valign": "vcenter",
    "border": 1,
    "bg_color": "#F5F5F5",
    "font_size": 24,
    "bold": True,
})

# Force vertical center on *every* format in the workbook
for fmt in wb.formats:
    fmt.set_align('vcenter')


# row_even = wb.add_format({"align": "left", "valign": "vcenter", "border": 1})
# row_odd  = wb.add_format({"align": "left", "valign": "vcenter", "border": 1, "bg_color": "#F5F5F5"})
# num_even = wb.add_format({"align": "right", "valign": "vcenter", "border": 1})
# num_odd  = wb.add_format({"align": "right", "valign": "vcenter", "border": 1, "bg_color": "#F5F5F5"})

_cell_fmt_cache = {}
_cell_fmt_cache = {}
def cell_fmt_for(style: dict):
    """
    Builds a composite XlsxWriter format from a style dict.
    Recognized keys: bg, font_color, bold, font_size, thick_border (bool)
    """
    key = (
        style.get("bg"),
        style.get("font_color"),
        bool(style.get("bold")),
        style.get("font_size", 11),
        bool(style.get("thick_border", False)),
    )
    if key in _cell_fmt_cache:
        return _cell_fmt_cache[key]

    borders = 2 if style.get("thick_border") else 1
    fmt = wb.add_format({
        "align": "center",
        "valign": "vcenter",
        "text_wrap": True,
        "bg_color": style.get("bg", DEFAULT_STYLE["bg"]),
        "font_color": style.get("font_color", DEFAULT_STYLE["font_color"]),
        "bold": bool(style.get("bold", False)),
        "font_size": style.get("font_size", 14),
        "left": borders, "right": borders, "top": borders, "bottom": borders,
    })
    _cell_fmt_cache[key] = fmt
    return fmt



# Title (optional)
weekday = DAY_DATE.strftime("%A")
nice_date = DAY_DATE.strftime("%B %d, %Y")
ws.merge_range(0, num_col, 0, last_time_col, f"Live Sports Schedule – {weekday} – {nice_date}", title_fmt) # wRITE hEADER rOW

# Header row for # and Channel + time columns
ws.write(1, num_col, "", num_hdr)
ws.write(1, chan_col, "", chan_hdr)
for i, lab in enumerate(COL_LABELS):
    ws.write(1, first_time_col + i, lab, header_fmt)

# Column widths
ws.set_column(num_col, num_col, 8)
ws.set_column(chan_col, chan_col, 20)
ws.set_column(first_time_col, last_time_col, 8)

# Set Row Height for all of the channel rows
total_channel_rows = sum(max(1, len(tracks)) for _, _, tracks in channel_tracks)
for r in range(2, 2 + total_channel_rows):
    ws.set_row(r, 42)   # default row height for event rows




# --------- RENDER: channels with as many tracks (rows) as needed ----------
current_row = 2
for ch_index, (ch_name, ch_num, tracks) in enumerate(channel_tracks):
    track_count = max(1, len(tracks))  # at least one row per channel
    for t_idx in range(track_count):
        shaded = ((current_row - 2) % 2 == 1)  # preserve global striping
        # Only show channel num/name on the first track row; blanks on overflow rows
        if t_idx == 0:
            ws.write(current_row, num_col, ch_num, num_odd if shaded else num_even)
            ws.write(current_row, chan_col, ch_name, row_odd if shaded else row_even)
        else:
            ws.write(current_row, num_col, "",     num_odd if shaded else num_even)
            ws.write(current_row, chan_col, "",    row_odd if shaded else row_even)

        # draw empty slot borders across the row
        for c in range(first_time_col, last_time_col + 1):
            ws.write(current_row, c, "", slot_border)

        # place events for this track (if any)
        if t_idx < len(tracks):
            for e in tracks[t_idx]:
                c0 = first_time_col + int(e["start_idx"])
                c1 = first_time_col + int(e["end_idx"]) - 1
                c1 = max(c0, min(c1, last_time_col))
                ws.merge_range(current_row, c0, current_row, c1, e["text"], cell_fmt_for(e["style"]))

        current_row += 1

# ========================= LEGEND RIBBON (single row, only used styles) =========================
# Collect unique legend items actually used today, preserve a sensible order
used = []                      # list of (name, style)
seen_labels = set()

# Priority: MSU, Big Ten, Top-25 first (if used), then base leagues by first appearance
PRIORITY = ("Michigan State", "Big Ten team", "Top-25 team")

# 1) Scan df in order of appearance
for r in df.itertuples():
    name = getattr(r, "legend_label", None)
    if not name:
        continue
    if name not in seen_labels:
        used.append((name, r.style))
        seen_labels.add(name)

# 2) Reorder so priority labels come first (if present)
def priority_key(item):
    name, _ = item
    return (0 if name in PRIORITY else 1, PRIORITY.index(name) if name in PRIORITY else 999)
used.sort(key=priority_key)

if used:
    # spacer row
    spacer_row = current_row
    ws.set_row(spacer_row, 6)
    legend_row = spacer_row + 1

    # compute per-item span across the time grid
    total_cols = (last_time_col - first_time_col + 1)
    n = len(used)
    # Minimum span to be readable
    min_span = 3
    span = max(min_span, total_cols // n) if n > 0 else total_cols

    # If rounding leaves free columns at the end, just let the last item absorb them
    col = first_time_col
    for i, (name, style_dict) in enumerate(used):
        c0 = col
        c1 = col + (span - 1)
        if i == n - 1:
            c1 = last_time_col  # stretch last swatch to the end
        c1 = min(c1, last_time_col)

        # The legend swatch uses the same cell format (bg/font/bold); borders follow the style (double if MSU)
        fmt = cell_fmt_for(style_dict)
        ws.merge_range(legend_row, c0, legend_row, c1, name, fmt)

        col = c1 + 1
        if col > last_time_col:
            break

    # make the legend row a bit taller
    ws.set_row(legend_row, 22)

    
    # ---- Page setup / print settings ----
    from xlsxwriter.utility import xl_range

    # last_row = current_row - 1              # last row we wrote
    # first_row = 0                           # include title/subheader
    # first_col = num_col                     # start at the "#" column
    # last_col  = last_day_col                # last day column
    # ws.print_area(first_row, first_col, last_row, last_col)

    # Page orientation and paper
    ws.set_landscape()          # Landscape
    ws.set_paper(1)             # 1 = Letter (8.5" x 11"). Use 9 for A4.

    # Margins Set to Max
    
    ws.set_margins(left=0.2, right=0.2, top=0.2, bottom=0.2)

    # Center on page (nice for single sheet handouts)
    ws.center_horizontally()
    ws.center_vertically()

    # Fit to a single printed page (1 page wide × 1 page tall)
    ws.fit_to_pages(1, 1)

    # Define the print area so the sheet opens ready to print one page
    

    # Optional: repeat header row (the row with "# / Channel / dates") on each printed page
    # (harmless even when we fit to one page)
    ws.repeat_rows(2, 2)  # zero-based row index; your headers are on row 2

    # Optional: show/hide gridlines in print (2 = hide on screen & print)
    ws.hide_gridlines(2)

    # Optional: header/footer
    ws.set_header('&L&"Calibri,Bold"&12StoryPoint – East Lansing'
                '&R&"Calibri"&10Printed &D')
    # ws.set_footer('&CPage &P of &N')



wb.close()
print("Wrote:", out_xlsx)


Wrote: output\SingleDay_2025-11-01.xlsx
